In [ ]:
import pyzbar.pyzbar as pyzbar
import cv2
from gtts import gTTS
from pygame import mixer
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
#pyzbar 바코드 인식, cv2 카메라 사용
#gtts 텍스트를 .mp3로
#pygame .mp3 파일 실행
#selenium, request, bs4 웹 크롤링에 사용

#웹 드라이버는 별도로 설치해야함
#웹 드라이버 다운로드 (https://chromedriver.chromium.org/downloads)

#***현재 버전에 맞는 크롤러 다운***

#크롬 버전 확인법
#우측 상단의 세로로 점 3개가 배열된 것 클릭 -> 도움말 -> chrome정보 

#카메라 실행
cap = cv2.VideoCapture(0)

score = 0
i = 0

#pygame 사용 전 필수(해주지 않으면 pygame사용 시 오류) 
mixer.init()

#카메라가 실행 된 동안
while(cap.isOpened()):
    ret, img = cap.read()

    if not ret:
        continue
        
    #img를 grayscale로 변경
    #gray이미지에서 pyzbar을 통해 바코드 검출
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    decoded = pyzbar.decode(gray)
    

    for d in decoded: 
        x, y, w, h = d.rect
        
        #첫 번째 때, 한 번만 실행
        if i == 0:
            #제일 처음 인식안 바코드와 바코드 타입(일반적인 바코드인지 QR코드인지)
            barcode_ori = d.data.decode("utf-8") 
            barcode_type = d.type

            #바코드가 감지되었다고 사용자에게 음성으로 알림
            mixer.music.load('barcode_detect.mp3')
            mixer.music.play()
            
            #i=1로 바꾸어서 
            i=1
        #실시간으로 감지한 바코드
        barcode_data = d.data.decode("utf-8")
  
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
        text = barcode_data + barcode_type
        cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
        #바코드에 사각형과 택스트 출력하는 것. 나중에 빼도 괜찮음
        
        
        if i>=1:
            if barcode_ori == barcode_data:
                score = score + 1
            i=i+1    #검색하고 한자리씩 지워주어야 함
    
    if i==30:
        if score >=25:
            try:
                #검색을 시작했다고 사용자에게 알려줌
                mixer.music.load("barcode_search.mp3")
                
                #웹 드라이버 사용
                #webdriver.Chrome(C:/testAI/~~~)은 다운받은 웹드라이버의 경로
                #현재 크롬의 버전과 웹 드라이버의 버전이 같아야 함
                #버전과 관련된 오류가 날 때는 사용중인 크롬의 버전과 웹 크롤러의 버전 확인
                #driver.get('크롤링할 사이트 주소')
                driver = webdriver.Chrome('C:/testAI/Web_Crawlling/chromedriver/chromedriver.exe')
                driver.get('https://www.consumer.go.kr/user/ftc/consumer/goodsinfo/57/selectGoodsInfoList.do?tabType=01')
                
                #검색창의 상태를 colum2로 변경하여 상품명 검색에서 바코드 검색으로 바꾸어줌
                search_box = driver.find_element_by_id("column2")
                search_box.click()
                
                #바코드 검색 후 바코드가 검색된 사이트 url가져온 후 드라이버 종료
                search_box = driver.find_element_by_id("search2")
                search_box.send_keys(barcode_ori)
                search_box.send_keys(Keys.RETURN)
                url = driver.current_url
                driver.quit()
                
                #크롤링을 통해서 가져온 url에서 약의 상표명을 가져옴
                result = requests.get(url = url)
                medicine_html = BeautifulSoup(result.content, "html.parser")
                medicine_name = medicine_html.find("tr").find("td").text
                
                #약 이름으로 tts제작 후 약의 상표명을 사용자에게 음성으로 알림
                tts = gTTS(text=medicine_name, lang='ko')
                tts.save("medicine_name.mp3")
                mixer.music.load("medicine_name.mp3")
                mixer.music.play()
                break
            
            except:
                #검색시에 오류가 나온 경우
                #보통 약의 바코드가 등록되지 않을 떄 크롤링 부분에서 오류 발생
                #약을 찾지 못한다고 사용자에게 음성으로 알림
                mixer.music.load("not_find.mp3")
                mixer.music.play()
                cap.release()
                cap.destroyAllWindows()
        
        else:
            #처음 찍은 바코드가 30번 중에 6번 이상 틀린 경우
            #다른 바코드 혹은 오류가 발생했다고 판단
            #에러가 발생했다고 사용자에게 알림
            mixer.music.load("barcode_error.mp3")
            mixer.music.play()



    cv2.imshow('img', img)

    #q를 누르면 종료
    #s를 누르면 일시정지
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    elif key == ord('s'):
        i += 1
        
#사용된 카메라를 종료해줌    
cap.release()
cv2.destroyAllWindows()